# Initialization

## Packages

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

from itertools import product

from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import roc_auc_score, roc_curve

## Environment

In [2]:
# Environment variables
PROJECTPATH = os.getenv('PROJECTPATH')

## Functions

In [3]:
def build_cluster_pairs(k):

    pairs = []
    for pair in product(k, k):
        pair = tuple(sorted(pair))
        if pair[0] != pair[1]:
            if pair not in pairs:
                pairs.append(pair)
                
    return pairs


def prepare_data(clusters, scores, demographics, nk = 2, threshold = 0.5, return_features = False):
    
    nk_col = 'nk{}'.format(nk)
    
    # Compute completion rate for the different assessments
    nparticipants = scores.shape[0]
    completion = dict()
    for col, vals in scores.items():
        if col != 'Subject_ID':
            completion[col] = vals.notna().sum()/nparticipants
            
    # Filter scales above completion threshold
    scales = [col for col, val in completion.items() if val > threshold]        
    
    # Get the scores for the subset of scales
    scores_subset = scores[scales].copy()
    scores_subset['Subject_ID'] = scores['Subject_ID']
    
    # Join scores and cluster information
    clusters_scores = clusters.copy()
    clusters_scores = (clusters_scores
     .rename(columns = {'ID':'file'})
     .loc[:, ['file', nk_col]]
     .merge(demographics, how = 'left', on = 'file'))

    # Filter for POND
    clusters_scores = clusters_scores.loc[clusters_scores['Dataset'] == 'POND']

    # Clean up IDs for merging
    clusters_scores['Subject_ID'] = (clusters_scores['Subject_ID']
                                     .str.replace('sub-', '')
                                     .astype(int))

    # Merge clusters to scores
    clusters_scores = (clusters_scores
                       .loc[:, ['Subject_ID', nk_col]]
                       .merge(scores_subset, on = 'Subject_ID', how = 'left'))

    # Keep only complete observations
    clusters_scores = clusters_scores.dropna()
    
    # Create the input matrix and binary targets
    X = clusters_scores.drop(['Subject_ID', nk_col], axis = 1)
    features = X.columns.to_list()
    X = X.to_numpy()
    y = np.array(clusters_scores[nk_col], dtype = int)-1
    
    if return_features:
        return X,y,features
    else:
        return X,y
    
    
def run_plsda(X, y):
    
    # Maximum number of possible components
    max_components = X.shape[1]-1
    
    # Range of model components
    component_range = range(2, max_components)

    # Iterate over model components
    auc = []
    for nc in component_range:

        # Initiatlize the PLSR module
        plsr = PLSRegression(n_components = nc, scale = True)

        # Fit the model to the data
        plsr.fit(X, y)

        # Predict cluster labels
        y_pred = plsr.predict(X)

        # Clamp the interval since this isn't a proper classifier
        y_pred[y_pred > 1.0] = 1.0
        y_pred[y_pred < 0] = 0

        # Compute AUC
        auc.append(roc_auc_score(y, y_pred))

    # Store AUC information in data frame
    df_auc = pd.DataFrame({'components':component_range, 'auc':auc})

    return df_auc
    

---
# Data preparation

In [4]:
# Parameter set ID
params_id = 700

# Output directory
output_dir = 'figure_supplementary'
output_dir = os.path.join(PROJECTPATH, 'figures', 'v3', output_dir)

# Input directories
registration_dir = 'data/human/registration/v3/'
pipeline_dir = 'data/human/derivatives/v3/'

registration_dir = os.path.join(PROJECTPATH, registration_dir)
pipeline_dir = os.path.join(PROJECTPATH, pipeline_dir, str(params_id))

# Demographics file
demographics = os.path.join(registration_dir, 'subject_info', 'demographics.csv')
demographics = pd.read_csv(demographics)

# POND clinical scores
scores = os.path.join(registration_dir, 'subject_info', 'POND', 'POND_clinical_scores_20230915.csv')
scores = pd.read_csv(scores)

# Cluster solutions
cluster_dir = os.path.join(pipeline_dir, 'clusters', 'resolution_3.0')
cluster_file = os.path.join(cluster_dir, 'clusters.csv')
clusters = pd.read_csv(cluster_file)

# Drop columns
cols_to_drop = ['Unnamed: 0', 'site', 'SUB_ID', 
                'DOB', 'PRIMARY_DIAGNOSIS', 
                'RESEARCH_CONFIRM_DIAG', 
                'HSHLD_INCOME_STD', 
                'PRMY_CGVR_STD',
               'SWANPDOC', 'TPOCSPDOC']
scores = scores.drop(cols_to_drop, axis = 1)

# Drop columns containing the following strings
strings_to_drop = ['NSI', 'ETHNCTY', 'EDUC']
for s in strings_to_drop:
    scores = scores.loc[:, ~scores.columns.str.contains(s)]

# Rename the subject ID column for merging
scores = scores.rename(columns = {'subject':'Subject_ID'})

# Assign NaN to missing values 999 code
for col, vals in scores.items():
    x = vals.copy()
    x[x == 999] = np.nan
    scores[col] = x

---
# Pairwise PLS-DA across cluster solutions

In [9]:
# Maximum number of cluster solutions
nk_max = 10

# Cluster solutions
nk_list = list(range(2, nk_max+1))

# Completion thresholds
thresholds = [0.6, 0.8]

# Iterate over cluster solutions
df_results = pd.DataFrame()
for nk in nk_list:
    
    # Number of clusters
    klist = list(range(1, nk+1))

    # Pairs of clusters
    kpairs = build_cluster_pairs(k = klist)

    # Iterate over cluster pairs
    for k in kpairs:

        # Get cluster IDs
        cluster_ids = ['{}-{}'.format(nk, ki) for ki in k]

        # Decrement cluster labels by 1
        k = [x-1 for x in k]

        # Iterate over completion thresholds
        for threshold in thresholds:

            # Get inputs and labels
            X,y = prepare_data(clusters = clusters, 
                               scores = scores, 
                               demographics = demographics,
                               nk = nk,
                               threshold = threshold, 
                               return_features = False)

            # Filter for participants in the clusters being compared
            ind_subset = np.isin(y, k)
            X = X[ind_subset,:]
            y = y[ind_subset]

            # Binarize labels
            y[y == k[0]] = 0
            y[y == k[1]] = 1
            
            # Run PLS-DA and catch instances that throw warnings
            with warnings.catch_warnings(record = True) as w:
                df_results_k = run_plsda(X = X, y = y)
                
            # Populate data frame instance
            df_results_k['nk'] = nk
            df_results_k['cluster_id_1'] = cluster_ids[0]
            df_results_k['cluster_id_2'] = cluster_ids[1]
            df_results_k['threshold'] = threshold
            df_results_k['participants'] = X.shape[0]
            df_results_k['participants_1'] = sum(y == 0)
            df_results_k['participants_2'] = sum(y == 1)
            df_results_k['features'] = X.shape[1]
            df_results_k['warning'] = True if len(w) > 0 else False
            
            # Concatenate data frames
            df_results = pd.concat([df_results, df_results_k])

# Reset index
df_results = df_results.reset_index(drop = True)

# Export results
outfile = 'plsda_results.csv'
outfile = os.path.join(output_dir, outfile)
df_results.to_csv(outfile, index = False)